In [1]:
import os
import gc
import time
import copy
import torch
import argparse
import numpy as np
from Model import *
from torch import optim
from DataLoader import *
from helperUtils import *
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision import transforms, utils
from torch.utils.data import DataLoader, Dataset
from tensorboard_logger import configure, log_value
from sklearn.metrics import normalized_mutual_info_score

In [2]:
## Loading the testing data.
testData = testDataSet('FRGC')

## Creating the test loader.
testLoader = DataLoader(testData, batch_size = 64, shuffle = True)

In [3]:
## Loading the trained model.
Model = torch.load('./TrainingResults/FRGC/PreTraining/Percentage10/λAnnealResults/AutoEncoderλAnneal.pkl', map_location = 'cpu' )

## Loading the cluster centers.
clusterCenters = torch.from_numpy(np.load('./TrainingResults/FRGC/PreTraining/Percentage10/λAnnealResults/clusterCenters.npy'))

## Directory for saving the original and reconstructed images.
saveDir = './TestingResults/FRGC/Images/'

In [5]:
## Creating placeholders for storing the predicted and true cluster labels.
yPredAll = []
yTrueAll = []

## Setting the AutoEncder to evaluation model.
Model.eval()

with torch.no_grad():
    
    for batchIndex, data in enumerate(testLoader):
        
        ## Loading the data batch.
        imgBatch, labelBatch = data
        imgBatchV, labelBatchV = Variable(imgBatch), Variable(labelBatch)
        ## imgBatchV, labelBatchV = Variable(imgBatch.cuda()), Variable(labelBatch.cuda())
        
        ## Performing the forward pass.
        encodings, reconstructions = Model(imgBatchV)
        
        ## Saving the original inputs as well as the reconstructions.
#         utils.save_image(imgBatch.data.cpu(), saveDir + str(batchIndex) + 'Original.png', nrow = 4)
#         utils.save_image(reconstructions.data.cpu(), saveDir + str(batchIndex) + 'Reconstructed.png', nrow = 4)
        
        ## Expanding the encodings and the cluster centers.
        encodingsExpand = encodings.unsqueeze(1).expand(encodings.size(0), 20, 32)
        clusterCentersExpand = clusterCenters.clone().unsqueeze(0).expand(encodings.size(0), 20, 32)
        
        ## Computing the distances of the encodings from the cluster centers.
        distMat = torch.pow(encodingsExpand - clusterCentersExpand, 2).sum(2)
        
        ## Computing the cluster center label having the minimum distance from the particular image encoding.
        _ , predClus = torch.min(distMat, dim = 1)
        
        ## Adding the predictions to the global list.
        yPredAll.extend(list(predClus.data.cpu().numpy()))
        
        ## Adding the true labels to the global list.
        yTrueAll.extend(list(labelBatchV.data.cpu().numpy()))

In [6]:
## Computing the test NMI and Purity.
testPurity = purityScore(yPredAll, yTrueAll)
testNMI = normalized_mutual_info_score(yPredAll, yTrueAll)

print(testPurity)
print(testNMI)

0.23076923076923078
0.23501780264222746


/Users/rahulbaboota/Documents/ClusteringGithub/venv/lib/python3.7/site-packages/sklearn/metrics/cluster/supervised.py:859: FutureWarning: The behavior of NMI will change in version 0.22. To match the behavior of 'v_measure_score', NMI will use average_method='arithmetic' by default.
  FutureWarning)
